# Importing Libraries

In [1]:
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split, WeightedRandomSampler, Subset
from torchvision import datasets, transforms
from collections import Counter
import numpy as np
from sklearn.model_selection import train_test_split
from torchvision.models import resnet18
from sklearn.metrics import f1_score, classification_report

# Importing Dataset

In [2]:
root_dir = "dataset"
train_ratio = 0.8
batch_size = 32
num_workers = 4

train_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

full_dataset = datasets.ImageFolder(root=root_dir)
targets = np.array(full_dataset.targets)
indices = np.arange(len(targets))

train_idx, val_idx = train_test_split(
    indices,
    test_size=1 - train_ratio,
    stratify=targets,
    random_state=42
)

train_dataset = Subset(datasets.ImageFolder(root=root_dir, transform=train_transform), train_idx)
val_dataset = Subset(datasets.ImageFolder(root=root_dir, transform=val_transform), val_idx)

train_targets = targets[train_idx]
class_counts = Counter(train_targets)
num_samples = len(train_dataset)

class_weights = {cls: num_samples / count for cls, count in class_counts.items()}
sample_weights = [class_weights[t] for t in train_targets]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=num_samples, replacement=True)

train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    sampler=sampler,
    num_workers=num_workers)
val_loader = DataLoader(
    val_dataset, 
    batch_size=batch_size, 
    shuffle=False,
    num_workers=num_workers)

print("Classes:", full_dataset.classes)
print("Train size:", len(train_dataset))
print("Validation size:", len(val_dataset))
print("Train class counts:", class_counts)

Classes: ['battery', 'biological', 'cardboard', 'clothes', 'glass', 'leafs', 'metal', 'paper', 'plastic_bag', 'plastic_bottle']
Train size: 52669
Validation size: 13168
Train class counts: Counter({np.int64(1): 12689, np.int64(3): 8522, np.int64(8): 8000, np.int64(7): 7107, np.int64(4): 4450, np.int64(5): 3595, np.int64(2): 2626, np.int64(9): 2421, np.int64(6): 1748, np.int64(0): 1511})


# Model Definition

In [3]:
num_classes = len(full_dataset.classes)

model = resnet18(weights="IMAGENET1K_V1")
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

epochs = 15

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = correct / total

    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == labels).sum().item()
            val_total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    val_loss /= val_total
    val_acc = val_correct / val_total

    f1_macro = f1_score(all_labels, all_preds, average='macro')
    print(
        f"Epoch [{epoch+1}/{epochs}] "
        f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | "
        f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f} | "
        f"F1 (macro): {f1_macro:.4f}")

print("\nFinal classification report:")
print(classification_report(all_labels, all_preds, target_names=full_dataset.classes))


Epoch [1/15] Train Loss: 0.3779 | Train Acc: 0.8742 | Val Loss: 0.1222 | Val Acc: 0.9613 | F1 (macro): 0.9476
Epoch [2/15] Train Loss: 0.2536 | Train Acc: 0.9163 | Val Loss: 0.1602 | Val Acc: 0.9493 | F1 (macro): 0.9281
Epoch [3/15] Train Loss: 0.2110 | Train Acc: 0.9294 | Val Loss: 0.0985 | Val Acc: 0.9683 | F1 (macro): 0.9566
Epoch [4/15] Train Loss: 0.1912 | Train Acc: 0.9356 | Val Loss: 0.1110 | Val Acc: 0.9639 | F1 (macro): 0.9564
Epoch [5/15] Train Loss: 0.1770 | Train Acc: 0.9408 | Val Loss: 0.0943 | Val Acc: 0.9707 | F1 (macro): 0.9596
Epoch [6/15] Train Loss: 0.1636 | Train Acc: 0.9454 | Val Loss: 0.1087 | Val Acc: 0.9681 | F1 (macro): 0.9550
Epoch [7/15] Train Loss: 0.1540 | Train Acc: 0.9481 | Val Loss: 0.0817 | Val Acc: 0.9743 | F1 (macro): 0.9668
Epoch [8/15] Train Loss: 0.1478 | Train Acc: 0.9504 | Val Loss: 0.0830 | Val Acc: 0.9740 | F1 (macro): 0.9652
Epoch [9/15] Train Loss: 0.1432 | Train Acc: 0.9521 | Val Loss: 0.0886 | Val Acc: 0.9727 | F1 (macro): 0.9625
Epoch [10/

# Exporting the Model

In [7]:
torch.save(model.state_dict(), "model.pth")

In [8]:
dummy_input = torch.randn(1, 3, 224, 224).to(device)
torch.onnx.export(
    model,
    dummy_input,
    "model.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=12
)

W1113 02:06:17.013000 4676 Lib\site-packages\torch\onnx\_internal\exporter\_compat.py:114] Setting ONNX exporter to use operator set version 18 because the requested opset_version 12 is a lower version than we have implementations for. Automatic version conversion will be performed, which may not be successful at converting to the requested version. If version conversion is unsuccessful, the opset version of the exported model will be kept at 18. Please consider setting opset_version >=18 to leverage latest ONNX features


[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `ResNet([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


The model version conversion is not supported by the onnxscript version converter and fallback is enabled. The model will be converted using the onnx C API (target version: 12).
Failed to convert the model to the target version 12 using the ONNX C API. The model was not modified
Traceback (most recent call last):
  File "d:\Projects\radya\garbage-classification\venv\Lib\site-packages\onnxscript\version_converter\__init__.py", line 127, in call
    converted_proto = _c_api_utils.call_onnx_api(
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Projects\radya\garbage-classification\venv\Lib\site-packages\onnxscript\version_converter\_c_api_utils.py", line 65, in call_onnx_api
    result = func(proto)
             ^^^^^^^^^^^
  File "d:\Projects\radya\garbage-classification\venv\Lib\site-packages\onnxscript\version_converter\__init__.py", line 122, in _partial_convert_version
    return onnx.version_converter.convert_version(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅
Applied 40 of general pattern rewrite rules.


ONNXProgram(
    model=
        <
            ir_version=10,
            opset_imports={'': 18},
            producer_name='pytorch',
            producer_version='2.9.0+cu126',
            domain=None,
            model_version=None,
        >
        graph(
            name=main_graph,
            inputs=(
                %"input"<FLOAT,[1,3,224,224]>
            ),
            outputs=(
                %"output"<FLOAT,[1,10]>
            ),
            initializers=(
                %"fc.bias"<FLOAT,[10]>{TorchTensor<FLOAT,[10]>(Parameter containing: tensor([ 0.0095, -0.0285,  0.0242, -0.0329,  0.0034,  0.0123,  0.0186, -0.0023, -0.0275,  0.0282], device='cuda:0', requires_grad=True), name='fc.bias')},
                %"conv1.weight"<FLOAT,[64,3,7,7]>{Tensor(...)},
                %"layer1.0.conv1.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
                %"layer1.0.conv2.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},
                %"layer1.1.conv1.weight"<FLOAT,[64,64,3,3]>{Tensor(...)},

# Validation

In [9]:
num_classes = len(full_dataset.classes)
model = resnet18(weights="IMAGENET1K_V1")
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

model.load_state_dict(torch.load("model.pth", map_location="cpu"))

model.eval()


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
import torch
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F

def predict_image(model, image_path, class_names):
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        ),
    ])

    image = Image.open(image_path).convert("RGB")
    input_tensor = transform(image).unsqueeze(0)

    model.eval()
    with torch.no_grad():
        outputs = model(input_tensor)
        probabilities = F.softmax(outputs, dim=1)[0]

    results = {class_names[i]: float(probabilities[i]) for i in range(len(class_names))}

    return results


In [11]:
class_names = full_dataset.classes
result = predict_image(model, "test/battery1.jpg", class_names)

top_class = max(result, key=result.get)
print(f"Predicted: {top_class} ({result[top_class]*100:.2f}%)")

for cls, prob in result.items():
    print(f"{cls}: {prob*100:.2f}%")

Predicted: battery (99.42%)
battery: 99.42%
biological: 0.00%
cardboard: 0.00%
clothes: 0.00%
glass: 0.00%
leafs: 0.00%
metal: 0.00%
paper: 0.58%
plastic_bag: 0.00%
plastic_bottle: 0.00%


In [ ]:
import os

correct = 0
total = 0

for filename in os.listdir("test"):
    if not filename.lower().endswith((".jpg", ".png", ".jpeg")):
        continue

    image_path = os.path.join("test", filename)

    # Extract label from filename (everything before the first digit)
    true_label = ''.join([c for c in filename if not c.isdigit()]).split('.')[0]

    result = predict_image(model, image_path, class_names)
    predicted_label = max(result, key=result.get)

    is_correct = predicted_label.lower() == true_label.lower()
    total += 1
    correct += int(is_correct)

    print(f"{filename}: predicted {predicted_label} ({result[predicted_label]*100:.2f}%) | true: {true_label} | {'✓' if is_correct else '✗'}")

accuracy = (correct / total) * 100 if total > 0 else 0
print(f"\nAccuracy: {accuracy:.2f}%")


battery1.jpg: predicted battery (99.42%) | true: battery | ✓
biological1.jpg: predicted biological (100.00%) | true: biological | ✓
cardboard1.jpg: predicted cardboard (95.86%) | true: cardboard | ✓
clothes1.jpg: predicted clothes (78.99%) | true: clothes | ✓
glass1.jpg: predicted glass (99.77%) | true: glass | ✓
metal1.jpg: predicted metal (100.00%) | true: metal | ✓
paper1.jpg: predicted biological (43.19%) | true: paper | ✗
plastic1.jpg: predicted metal (59.98%) | true: plastic | ✗

Accuracy: 75.00%


: 